In [1]:
import pandas as pd 
from pandas import datetime
from pandas import DataFrame
from statsmodels.tsa.arima_model import ARMA
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import numpy as np
import random
import warnings
import quandl #loading quandl package for data gathering
from pandas import Series #to create required Series to be added to certain dataframes
from math import sqrt  #in order to compute RMS values of observed and predicted values
from statsmodels.tsa.ar_model import AR #to import Auto-Regression model required for predicting stock data
from sklearn.metrics import mean_squared_error #to find MSE of expected an predicted values
from matplotlib import pyplot #for interactive plotting of required graphs
pyplot.style.use('fivethirtyeight') #maintain uniform style of visualisation
quandl.ApiConfig.api_key = "cYf2RmzUcPBMGfsBrxhR" #Quandl API key in order to obtain data from web

C:\Users\HP\Anaconda3\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [ ]:
#gathering data from Quandl
icici = quandl.get('NSE/ICICIBANK.5',start_date='2017-01-01',end_date='2017-12-26')
bobr = quandl.get('NSE/BANKBARODA.5',start_date='2017-01-01',end_date='2017-12-26')
boi = quandl.get('NSE/BANKINDIA.5',start_date='2017-01-01',end_date='2017-12-26' )
canbk = quandl.get('NSE/CANBK.5',start_date='2017-01-01',end_date='2017-12-26' )
fedbk = quandl.get('NSE/FEDERALBNK.5',start_date='2017-01-01',end_date='2017-12-26' )
axis = quandl.get('NSE/AXISBANK.5',start_date='2017-01-01',end_date='2017-12-26' )
hdfc = quandl.get('NSE/HDFCBANK.5',start_date='2017-01-01',end_date='2017-12-26' )
indus = quandl.get('NSE/INDUSINDBK.5',start_date='2017-01-01',end_date='2017-12-26' )
kotak = quandl.get('NSE/KOTAKBANK.5' ,start_date='2017-01-01',end_date='2017-12-26')
pnb = quandl.get('NSE/PNB.5',start_date='2017-01-01',end_date='2017-12-26' )
sbin = quandl.get('NSE/SBIN.5',start_date='2017-01-01',end_date='2017-12-26' )
yesbk = quandl.get('NSE/YESBANK.5',start_date='2017-01-01',end_date='2017-12-26' )

In [ ]:
#creating a dictionary of all the stocks in a portfolio
d1 = {'ICICI':icici['Close'], 'BOB':bobr['Close'],'BOI':boi['Close'], 'CANBK':canbk['Close'],'FEDBK':fedbk['Close'], 
     'AXIS':axis['Close'],'HDFC':hdfc['Close'],'INDUSIND':indus['Close'],'KOTAK':kotak['Close'], 'PNB':pnb['Close'],
     'SBI':sbin['Close'], 'YESBANK':yesbk['Close']} 

df1 = pd.DataFrame(data=d1) #creating the dataframe
df1 #printing the dataframe

In [ ]:
df1.to_csv("closing prices.csv") #writing the dataframe to a csv file

In [ ]:
keys=[] #blank list to assign all stock names
keys=df1.keys() #appending all stock names from dataframe to the list

for i in range(len(df1.columns)): #loop to iterate over the stock names in the dataframe
    df1[keys[i]].plot() #plotting the closing prices of each stock
    
pyplot.xlim([df1.index[0],df1.index[-1]]) #assigning index values(date) on x-axis
pyplot.legend(loc="best") #assigning the legend location to best suit the plot
pyplot.show() #displaying the plot of closing prices

In [ ]:
#declaring function to predict values
def predict(coef, history): 
    yhat = 0.0 #initialising variable to store predicted value
    
    for i in range(1, len(coef)+1): #loop to iterate over model co-efficient parameters
        yhat += coef[i-1] * history[-i] #calculating predicted value
    return yhat #returning Predicted variable to be computed in the model

# AR MODEL

In [ ]:
#function to perform AR calculation
def autoreg(ar):
    global ar_mse #variable 'ar_mse' stores the Mean Squared Error between the Test Data and Predicted Values
    global ard #variable to create a dataframe of test and predicted values
    size = int(len(ar)/2) #Defining size of Train and Test Data
    ar_train, ar_test = ar[0:size], ar[size:] #Assigning range of values for Train and Test Data
    ar_history = [ar for ar in ar_train] #list comprehension for iterating through each element in train data
    ar_predictions = list() # list to store Predicted Values
    for t in range(len(ar_test)): #loop to iterate through the test data
        ar_model = ARMA(ar_history, order=(1,0)) #Creating AR Model
        ar_model_fit = ar_model.fit(trend='nc', disp=0) #obtaining the model values
        ar_coef = ar_model_fit.arparams #computing AR parameters from the model
        ar_yhat = predict(ar_coef, ar_history) #running function to generate predicted value
        ar_predictions.append(ar_yhat) #appending predicted values to the predictions list
        ar_obs = ar_test[t] #assigning observed value from test data to variable 'ar_obs'
        ar_history.append(ar_obs) #appending test data to the train data
        print('>predicted=%.3f, expected=%.3f' % (ar_yhat, ar_obs)) #printing the expected value and its corresponding test value
    ar_mse=mean_squared_error(ar_test, ar_predictions) #calculating Mean Squared Error between Observed and Predicted Values
    ard=pd.DataFrame({'Expected':ar_test,'Predicted':ar_predictions}) #creating a dataframe of observed and predicted values
    print('Test Root Mean Squared Error: %.3f' % sqrt(ar_mse)) #Printing the computed Root Mean Squared Error
    print(ar_model_fit.summary()) #Displaying the summary of AR Model Parameters
    pyplot.plot(ar.index[len(ar_test)-1:],ar_test, color='black',label='test') #plotting the Observed Values
    pyplot.plot(ar.index[len(ar_test)-1:],ar_predictions, color='red',label='predictions') #plotting the Predicted values
    pyplot.legend(loc='best') #Assigning location of graph legend to be best suited
    pyplot.title('Auto Rergression Model') #Title Of Graph
    pyplot.show() #Displaying the AR Model plot
    return autoreg #returning the entire function to be called by passing necessary Dataset

# MA MODEL

In [ ]:
#function to perform MA calculation
def movavg(ma):
    global ma_mse #variable 'ma_mse' stores the Mean Squared Error between the Test Data and Predicted Values
    global mad #variable to create a dataframe of test and predicted values
    size = int(len(ma)/2) #Defining size of Train and Test Data
    ma_train, ma_test = ma[0:size], ma[size:] #Assigning range of values for Train and Test Data
    ma_history = [ma for ma in ma_train] #list comprehension for iterating through each element in train data
    ma_predictions = list() # list to store Predicted Values
    for t in range(len(ma_test)): #loop to iterate through the test data
        ma_model = ARMA(ma_history, order=(0,1)) #Creating AR Model
        ma_model_fit = ma_model.fit(trend='nc', disp=False) #obtaining the model values
        ma_coef = ma_model_fit.maparams #computing MA parameters from the model
        ma_yhat = predict(ma_coef, ma_history) #running function to generate predicted value
        ma_predictions.append(ma_yhat) #appending predicted values to the predictions list
        ma_obs = ma_test[t] #assigning observed value from test data to variable 'ma_obs'
        ma_history.append(ma_obs) #appending test data to the train data
        print('>predicted=%.3f, expected=%.3f' % (ma_yhat, ma_obs)) #printing the expected alue and its corresponding test value
    ma_mse = mean_squared_error(ma_test, ma_predictions) #calculating Mean Squared Error between Observed and Predicted Values
    mad=pd.DataFrame({'Expected':ma_test,'Predicted':ma_predictions}) #creating a dataframe of observed and predicted values
    print('Test Root Mean Squared Error: %.3f' % sqrt(ma_mse)) #Printing the computed Root Mean Squared Error
    print(ma_model_fit.summary()) #Displaying the summary of MA Model Parameters
    pyplot.plot(icici.index[len(ma_test)-1:],ma_test, color='black',label='test') #plotting the Observed Values
    pyplot.plot(icici.index[len(ma_test)-1:],ma_predictions, color='red',label='predictions') #plotting the Predicted Values
    pyplot.legend(loc='best') #Assigning location of graph legend to be best suited
    pyplot.title('Moving Average Model') #Title Of Graph
    pyplot.show() #Displaying the MA Model plot
    return movavg #returning the entire function to be called by passing necessary Dataset

# ARMA MODEL

In [ ]:
#declaring function to compute successive differences of 2 elements within dataset
def difference(dataset):
    diff = list() #creating list to store the difference of successive elements in test data
    
    for i in range(1, len(dataset)): #loop to iterate over each element in the dataset
        value = dataset[i] - dataset[i - 1] #computing difference of succesive values
        diff.append(value) #appending difference value to list
    return np.array(diff) #returning the list of difference values as array

In [ ]:
#function to perform ARMA calculation
def armamod(arma):
    global arma_mse,armad
    arma_size = int(len(arma)/2) #Defining size of Train and Test Data
    arma_train, arma_test = arma[0:arma_size], arma[arma_size:] #Assigning range of values for Train and Test Data
    arma_history = [arma for arma in arma_train] #list comprehension for iterating through each element in train data
    arma_predictions = list() # list to store Predicted Values
    for t in range(len(arma_test)):
        arma_model = ARMA(arma_history, order=(1,0)) #Creating ARMA Model
        arma_model_fit = arma_model.fit(disp=0) #obtaining the model values
        ar_coef = arma_model_fit.arparams #computing AR parameters from the model
        ma_coef = arma_model_fit.maparams #computing MA parameters from the model
        arma_resid = arma_model_fit.resid #obtaining residual parameters from ARMA model
        arma_diff = difference(arma_history) #running the function to obtain difference between successive values
        arma_yhat = predict(ar_coef, arma_history)+predict(ma_coef, arma_diff) #computing the predicted values
        arma_predictions.append(arma_yhat) #appending predicted values to the predictions list
        arma_obs = arma_test[t] #assigning observed value from test data to variable 'arma_obs'
        arma_history.append(arma_obs) #appending test data to the train data
        print('>predicted=%.3f, expected=%.3f' % (arma_yhat, arma_obs)) #printing the expected value and its corresponding test value
    arma_mse = mean_squared_error(arma_test, arma_predictions)  #calculating Mean Squared Error between Observed and Predicted Values
    armad=pd.DataFrame({'Expected':arma_test,'Predicted':arma_predictions}) #creating a dataframe of observed and predicted values
    print('Test Root Mean Squared Error: %.3f' % sqrt(arma_mse)) #Printing the computed Root Mean Squared Error
    print(arma_model_fit.summary()) #Displaying the summary of ARMA Model Parameters
    pyplot.plot(icici.index[len(arma_test)-1:],arma_test, color='black',label='test') #plotting the Observed Values
    pyplot.plot(icici.index[len(arma_test)-1:],arma_predictions, color='red',label='predictions') #plotting the Predicted Values
    pyplot.legend(loc='best') #Assigning location of graph legend to be best suited
    pyplot.title('ARMA Model') #Title Of Plot
    pyplot.show() ##Displaying the ARMA Model plot with Observed and Predicted Values
    return armamod #returning the entire function to be called by passing necessary Dataset

# ARIMA MODEL

In [ ]:
# evaluate an ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
    train_size = int(len(X)/2) # prepare training dataset
    train, test = X[0:train_size], X[train_size:] #dividing the data into train and test data
    history = [x for x in train] #list comprehension for iterating through each element in train data
    
    predictions = list() # list to store Predicted Values
    for t in range(len(test)): #loop to iterate through the test data
        model = ARIMA(history, order=arima_order) #creating ARIMA model
        model_fit = model.fit(disp=0) #obtaining the model values
        yhat = model_fit.forecast()[0] #predicting the forecast for 1 step
        predictions.append(yhat) #appending predicted values to the predictions list
        history.append(test[t]) #appending test data to the train data
        
    error = np.sqrt(mean_squared_error(test, predictions)) # calculate out of sample error
    return error #returning error to be passed on the subsequent function

In [ ]:
# evaluate combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values: #loop to iterate through all values of AR order in ARIMA Model
        for d in d_values: #loop to iterate through all values of difference degree in ARIMA Model
            for q in q_values: #loop to iterate through all values of MA order in ARIMA Model
                order = (p,d,q) #variable 'order' takes a tuple of the corresponding p,d,q order for the ARIMA model
                try: #try block to assess the lowest MSE and best ARIMA order
                    mse = evaluate_arima_model(dataset, order) #variable 'mse' stores the MSE obtained from the previous function
                    if mse < best_score: #condition to check for lowest Mean Squared Error
                        best_score, best_cfg = mse, order #variable 'best_score' stores lowest MSE and best_cfg stores the corresponding ARIMA order for the lowest MSE
                        print('ARIMA%s MSE=%.3f' % (order,mse)) #to print the iterated ARIMA orders with their corresponding MSE
                except:
                    continue
    return best_cfg #returns the ARIMA order for lowest MSE

In [ ]:
#function to perform ARIMA calculation
def arimamod(arima): 
    global arima_mse #variable 'arima_mse' stores the Mean Squared Error between the Test Data and Predicted Values
    global arimad #variable to create a dataframe of test and predicted values
    arima_size = int(len(arima)/2) #Defining size of Train and Test Data
    arima_train, arima_test = arima[0:arima_size], arima[arima_size:] #Assigning range of values for Train and Test Data
    arima_history = [arima for arima in arima_train] #list comprehension to assess each element in train data
    arima_predictions = list() #declaring a list to store the predicted values after ARIMA predictions
    order=evaluate_models(arima, p_values, d_values, q_values) #running function to obtain best ARIMA order
    for t in range(len(arima_test)): #loop to iterate through the test data
        arima_model = ARIMA(arima_history, order)#Creating Arima Model
        arima_model_fit = arima_model.fit(disp=0) #Obtaining Model Parameters
        ar_coef= arima_model_fit.arparams #computing AR parameters from the model
        ma_coef = arima_model_fit.maparams #computing MA parameters from the model
        arima_resid = arima_model_fit.resid #obtaining residual parameters from ARMA model
        arima_diff = difference(arima_history) #running the function to obtain difference between successive values
        arima_yhat = arima_history[-1]+ predict(ar_coef, arima_diff) + predict(ma_coef, arima_resid) #computing the predicted values
        arima_predictions.append(arima_yhat)#appending predicted values to the predictions list
        arima_obs = arima_test[t] #assigning observed value from test data to variable 'arma_obs'
        arima_history.append(arima_obs) #appending test data to the train data
        print('>predicted=%.3f, expected=%.3f' % (arima_yhat, arima_obs))
    arima_mse = mean_squared_error(arima_test, arima_predictions)
    arimad=pd.DataFrame({'Expected':arima_test,'Predicted':arima_predictions})
    print('Test Root Mean Squared Error: %.3f' % sqrt(arima_mse)) #Printing the computed Root Mean Squared Error
    print(arima_model_fit.summary()) #Displaying the summary of ARIMA Model Parameters
    pyplot.plot(icici.index[len(arima_test)-1:],arima_test, color='black',label='test') #plotting the Observed Values
    pyplot.plot(icici.index[len(arima_test)-1:],arima_predictions, color='red',label='predictions')  #plotting the Predicted Values
    pyplot.legend(loc='best') #Assigning location of graph legend to be best suited
    pyplot.title('ARIMA Model') #Title Of Plot
    pyplot.show() #Displaying the ARIMA Model plot with Observed and Predicted Values
    print(order) #Displaying the best order with lowest RMSE for the ARIMA model
    return arimamod #returning the entire function to be called by passing necessary Dataset

In [ ]:
#ARIMA models are denoted with the notation ARIMA(p, d, q). 
#Together these three parameters account for seasonality, trend, and noise in datasets:

p_values = range(0, 4) #range to describe AR order of ARIMA Model
d_values = range(0, 4) #range of degree of differencing
q_values = range(0, 4) #range to describe MA order of ARIMA Model
warnings.filterwarnings("ignore") #to ignore warnings as it can give rise to errors

In [ ]:
arm=[] #list to store RMSE values of all stocks obtained from AR Model
mam=[] #list to store RMSE values of all stocks obtained from MA Model
armam=[] #list to store RMSE values of all stocks obtained from ARMA Model
arimam=[] #list to store RMSE values of all stocks obtained from ARIMA Model

for i in range(len(df1.columns)): #loop to iterate through all stocks in the portfolio
    
    autoreg(df1[keys[i]]) #Running AR Model for all the stocks
    arm.append('%.3f'%sqrt(ar_mse)) #appending all RMSE values of the different stocks from AR Model to list 'arm'
    #Create CSV file for predicted values of the corresponding model using the name of the corresponding stock as part of filename
    with open('AR of ' + str(df1.columns[i]) + '.csv', 'w') as csvfile:
        ard.to_csv(csvfile) #writing the dataframe of observed and predicted values from AR Model to CSV file
    
    movavg(df1[keys[i]]) #Running MA Model for all the stocks
    mam.append('%.3f'%sqrt(ma_mse)) #appending all RMSE values of the different stocks from MA Model to list 'mam'
    #Create CSV file for predicted values of the corresponding model using the name of the corresponding stock as part of filename
    with open('MA of ' + str(df1.columns[i]) + '.csv', 'w') as csvfile:
        mad.to_csv(csvfile) #writing the dataframe of observed and predicted values from AR Model to CSV file
        
    armamod(df1[keys[i]]) #Running ARMA Model for all the stocks
    armam.append('%.3f'%sqrt(arma_mse)) #appending all RMSE values of the different stocks from ARMA Model to list 'armam'
    #Create CSV file for predicted values of the corresponding model using the name of the corresponding stock as part of filename
    with open('ARMA of ' + str(df1.columns[i]) + '.csv', 'w') as csvfile:
        armad.to_csv(csvfile) #writing the dataframe of observed and predicted values from ARMA Model to CSV file
    
    arimamod(df1[keys[i]]) #Running the ARIMA Model for all the stocks
    arimam.append('%.3f'%sqrt(arima_mse)) #appending all RMSE values of the different stocks from ARIMA Model to list 'arimam'
    #Create CSV file for predicted values of the corresponding model using the name of the corresponding stock as part of filename
    with open('ARIMA of ' + str(df1.columns[i]) + '.csv', 'w') as csvfile:
        arimad.to_csv(csvfile) #writing the dataframe of observed and predicted values from ARIMA Model to CSV file

In [ ]:
for i in range(len(df1.columns)):
    arimamod(df1[keys[i]]) #Running the ARIMA Model for all the stocks
    arimam.append('%.3f'%sqrt(arima_mse)) #appending all RMSE values of the different stocks from ARIMA Model to list 'arimam'
    #Create CSV file for predicted values of the corresponding model using the name of the corresponding stock as part of filename
    with open('ARIMA of ' + str(df1.columns[i]) + '.csv', 'w') as csvfile:
        arimad.to_csv(csvfile) #writing the dataframe of observed and predicted values from ARIMA Model to CSV file

In [ ]:
armse=[] #list to store RMSE values of all stocks obtained from AR Model in float format
mamse=[] #list to store RMSE values of all stocks obtained from MA Model in float format
armamse=[] #list to store RMSE values of all stocks obtained from ARMA Model in float format
arimamse=[] #list to store RMSE values of all stocks obtained from ARIMA Model in float format

for i in arm: #loop to iterate through all elements in list 'arm'
    armse.append(float(i)) #converting string value to float and appending it to a new list 'armse'
print(armse)    

for i in mam: #loop to iterate through all elements in list 'mam'
    mamse.append(float(i)) #converting string value to float and appending it to a new list 'mamse'
print(mamse)   

for i in armam: #loop to iterate through all elements in list 'armam'
    armamse.append(float(i)) #converting string value to float and appending it to a new list 'armamse'
print(armamse)

for i in arimam: #loop to iterate through all elements in list 'arimam'
    arimamse.append(float(i)) #converting string value to float and appending it to a new list 'arimamse'
print(arimamse)

In [ ]:
print('RMS Error Values of all Stocks by different models')
#Creating array for comparing RMSE values of all stocks from all models
re=np.column_stack((armse,mamse,armamse,arimamse))
rt=pd.DataFrame(re) #Converting array to dataframe
rt.index=df1.keys() #Assigning Index/Row values to name of corresponding Stock Name
rt.columns=list(['AR','MA','ARMA','ARIMA']) #Assigning column names to the corresponding models
rt #displaying the dataframe

In [ ]:
returns = df1.pct_change() #percentage change matrix
#calculate mean daily return and covariance of daily returns
mean_daily_returns = returns.mean()
cov_matrix = returns.cov()
 
#set number of runs of random portfolio weights
num_portfolios = len(df1)
 
#set up array to hold results
#We have increased the size of the array to hold the weight values for each stock
results = np.zeros((4+len(df1.columns)-1, num_portfolios)) #array initialised to contain only zeros

for i in range(num_portfolios):
    weights = np.array(np.random.uniform(-0.3,0.3,12)) #assigning uniform weights to all stocks
    
    #calculate annualised portfolio return
    portfolio_return = round(np.sum(mean_daily_returns * weights) * 252,2)
    #calculate annualised portfolio volatility
    portfolio_std_dev = round(np.sqrt(np.dot(weights.T,np.dot(cov_matrix, weights))) * np.sqrt(252),2)
 
    #store results in results array
    results[0,i] = portfolio_return
    results[1,i] = portfolio_std_dev
    
    #store Sharpe Ratio (return / volatility) - risk free rate element excluded for simplicity
    results[2,i] = results[0,i] / results[1,i]
    
    #iterate through the weight vector and add data to results array
    for j in range(len(weights)):
        results[j+3,i] = weights[j]
 
#convert results array to Pandas DataFrame
results_frame = pd.DataFrame(results.T,columns=['returns','volatality','sharpe',df1.columns[0],df1.columns[1],df1.columns[2],
                                                df1.columns[3],df1.columns[4],df1.columns[5],df1.columns[6],df1.columns[7],
                                                df1.columns[8],df1.columns[9],df1.columns[10],df1.columns[11]])
results_frame

In [ ]:
#locate position of portfolio with highest Sharpe Ratio
max_sharpe_port = results_frame.iloc[results_frame['sharpe'].idxmax()]
#locate positon of portfolio with minimum standard deviation
min_vol_port = results_frame.iloc[results_frame['volatality'].idxmin()]
 
#create scatter plot coloured by Sharpe Ratio
pyplot.scatter(results_frame.volatality,results_frame.returns,c=results_frame.sharpe,cmap='RdYlBu')
pyplot.xlabel('Volatility')
pyplot.ylabel('Returns')
pyplot.colorbar()
#plot red star to highlight position of portfolio with highest Sharpe Ratio
pyplot.scatter(max_sharpe_port[1],max_sharpe_port[0],marker=(5,1,0),color='r',s=200)
#plot green star to highlight position of minimum variance portfolio
pyplot.scatter(min_vol_port[1],min_vol_port[0],marker=(5,1,0),color='g',s=200)
pyplot.show()

In [ ]:
print(max_sharpe_port)
print(min_vol_port)

In [ ]:
x = df1.std()
y = df1.mean()
x = [i/max(x) for i in x]
y= [(i-np.mean(y))/max(y) for i in y]

In [ ]:
shrat=[]
for i,j in zip(x,y):
    shrat.append(j/i)
ind=shrat.index(max(shrat))
print(shrat)
print('Best Sharpe Ratio belongs to',df1.columns[ind] )

In [ ]:
pyplot.scatter(x, y)
for index, symbol in enumerate(df1.columns):
    pyplot.annotate(symbol, (x[index], y[index]))

# Title and axis
#pyplot.ylim([-1,1])
pyplot.xlabel('Risk')
pyplot.ylabel('Expected Return')
pyplot.title('Expected Return versus Risk')
# Save the plot
pyplot.savefig('expected_return_vs_risk.png')
pyplot.show()

results_frame.index=df1.index
results_frame.to_csv("Results Frame.csv")